In [1]:
import datetime as dt 
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon, box
from shapely.ops import orient
import contextily as cnx
import rasterio
import sliderule
import tqdm
from tqdm import tqdm
import numpy as np

import os
import sys


In [2]:
#Uploading the Fire shapefile (boolean -> fire=1 , no fire=0). 
# It shows accumulated fire scars from 2019 - 2024. 
fire = gpd.read_file('Fire_2019_2024_vector.shp')

# Keep only burned areas (class 1)
fire_scars = fire[fire['Fire'] == 1] 

#Checking for the CRS (make sure it matches the one from fishnet)
#fire.crs

In [3]:
#Uploading the Arc of Deforestation outline
gf = gpd.read_file('AoD.geojson')

# Check CRS
#gf.crs

#gf.explore()

In [4]:
# Get bounding box coordinates
minx, miny, maxx, maxy = gf.total_bounds

# Define grid size (adjust depending on CRS: degrees for EPSG:4326, meters for projected) 
cell_width = 0.5 # change this to control grid size 
cell_height = 0.5

# Generate rows and columns 
cols = np.arange(minx, maxx, cell_width) 
rows = np.arange(miny, maxy, cell_height) 

# Create polygons for each grid cell 
grid_cells = [] 
for x in cols: 
    for y in rows: 
        cell = box(x, y, x + cell_width, y + cell_height) 
        grid_cells.append(cell) 
        
# Convert to GeoDataFrame 
fishnet = gpd.GeoDataFrame({'geometry': grid_cells}, crs=gf.crs).drop_duplicates('geometry') 

fishnet = gpd.sjoin(fishnet, gf, how='inner').drop_duplicates('geometry') 

fishnet['tile_id']=fishnet.index

#ax = fishnet.boundary.plot() 
#sample = fishnet.iloc[[23]] 
#sample.geometry.plot(color='red', edgecolor='black', ax = ax, alpha=0.5) 
#gf.plot(ax=ax, color = 'yellow', alpha = 0.2) 

fishnet.explore()

In [5]:
from sliderule import gedi
gedi.init(verbose=True)

In [6]:
results = []  # to store outputs per tile
for idx, tile in tqdm(fishnet.iterrows(), total=len(fishnet), desc="Processing tiles"):
    tile_id = tile['tile_id']
    sq = gpd.GeoDataFrame({'geometry':[tile.geometry]}, crs=fishnet.crs)

    # Ensure CRS is lat/lon
    sq = sq.to_crs(4326)

    # Simplify to avoid retry errors
    sq['geometry'] = sq['geometry'].simplify(0.001, preserve_topology=True)

    # ===> GEDI query for the tile 
    parms = {
        "poly": sliderule.toregion(sq)["poly"],
        "degrade_filter": True,
        "l2_quality_filter": True,
        "surface_filter": True
    }
    rsps = gedi.gedi04ap(parms)

    # If time is in the index, recover it as a column
    rsps['gedi_time'] = rsps.index 
    # Ensure it is a proper datetime object (if not already)
    rsps['gedi_time'] = pd.to_datetime(rsps['gedi_time'])
    # Apply filters only if the columns exist
    if "sensitivity" in rsps.columns:
        rsps = rsps[rsps["sensitivity"] > 0.98]
    if "beam" in rsps.columns:
        rsps = rsps[rsps["beam"].isin([5,6,8,11])]

    # --- Buffer GEDI shots ---
    rsps_reprojected = rsps.to_crs(5880)  # pick projected CRS
    rsps_buffer_reprojected = rsps_reprojected.copy()
    rsps_buffer_reprojected['geometry'] = rsps_reprojected.buffer(17.6)
    rsps_shot = rsps_buffer_reprojected.to_crs(4326)

        # --- Clip fire scars for this tile ---
    fire_tile = fire_scars.clip(sq)

    # --- Intersect GEDI shots with fire scars ---
    rsps_fire = gpd.sjoin(
        rsps_shot,  # keep gedi_time + agbd
        fire_tile[['geometry']],
        how='inner',
        predicate='within'
    )

    # Save results (could append GeoDataFrame or summary stats)
    results.append({
        "tile_id": tile_id,
        "gedi_fire": rsps_fire
    })

# Optionally combine into one GeoDataFrame
all_rsps_fire = gpd.GeoDataFrame(
    pd.concat([r["gedi_fire"] for r in results], ignore_index=True),
    crs=4326
)

Processing tiles:   7%|▋         | 24/342 [00:25<05:39,  1.07s/it] 


KeyboardInterrupt: 

In [8]:
rsps.head()

,geometry,gedi_time


In [7]:
all_rsps_fire.to_file(
    "Gedi_fire_SlideRule.gpkg",
    layer="Gedi_fire_test1",
    driver="GPKG"
)